<a href="https://colab.research.google.com/github/janedewes/Py-MachineLearning/blob/master/StreamDeDadosTwitterVfinal.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Stream de Dados do Twitter com MongoDB, Pandas e Scikit Learn

- Projeto: coletar dados do twitter, armazenar no MongoDB(orientado a documento), fazer análise com Pandas e Scikit Learn.

- Fazer um Text Mining, que é uma das aplicações de técnicas de processamento de linguagem natural e métodos analíticos para entrair informações relevantes de dados de texto.

- Criar uma API(App Programming Interface) para obter dados do Twitter, neste caso será usado o Twitter Streaming API, para fazer os downloads do twitter. 

## Preparando a Conexão com o Twitter

In [34]:
# Instala o pacote tweepy - entre py e twitter (! será executado no SO)
!pip install tweepy

In [35]:
# Importando os módulos Tweepy, Datetime e Json
from tweepy.streaming import StreamListener # ouvinte de twitter
from tweepy import OAuthHandler #autenticação com twitter
from tweepy import Stream # operar com stream
from datetime import datetime 
import json

In [36]:
# 1. Adicione aqui sua Consumer Key
consumer_key = ""

In [37]:
# 2. Adicione aqui sua Consumer Secret 
consumer_secret = ""

In [38]:
# 3. Adicione aqui seu Access Token
access_token = ""

In [39]:
# 4. Adicione aqui seu Access Token Secret
access_token_secret = ""

In [40]:
# Criando as chaves de autenticação do twitter
auth = OAuthHandler(consumer_key, consumer_secret)

In [41]:
# como a chave acima, autenticação do twitter
auth.set_access_token(access_token, access_token_secret)

In [24]:
# Criando uma classe para capturar os stream de dados do Twitter e 
# armazenar no MongoDB
class MyListener(StreamListener):
    def on_data(self, dados): #coleta os dados
        tweet = json.loads(dados) #converte para json
        created_at = tweet["created_at"] # daqui para baixo (3), as colunas que serão coletadas (retiradas da documentação da API do twitter, informando quais infos podem ser coletadas)
        id_str = tweet["id_str"]
        text = tweet["text"]
        obj = {"created_at":created_at,"id_str":id_str,"text":text,} # cria objeto
        tweetind = col.insert_one(obj).inserted_id # insere no Mongo DB
        print (obj)
        return True

In [42]:
# Criando o objeto mylistener - instanciar um objeto dessa classe
mylistener = MyListener()

In [43]:
# Criando o objeto mystream (usando a fç Stream)
mystream = Stream(auth, listener = mylistener)

## Preparando a Conexão com o MongoDB


In [55]:
!pip install pymongo

In [72]:
import pymongo

In [73]:
# Importando do PyMongo o módulo MongoClient
from pymongo import MongoClient   

In [74]:
# Criando a conexão ao MongoDB
client = MongoClient('localhost', 27017)

In [75]:
# Criando o banco de dados twitterdb
db = client.novotwitterdb

In [76]:
# Criando a collection "col" (coleção é uma espécie de tabela)
col = db.tweets 

In [77]:
# Criando uma lista de palavras chave para buscar nos Tweets
keywords = ['Leeds', 'Bielsa', 'Liverpool', 'Firmino']

## Coletando os Tweets

## -> Pressione o botão Stop na barra de ferramentas para encerrar a captura dos Tweets

In [84]:
# Iniciando o filtro e gravando os tweets no MongoDB
mystream.filter(track=keywords)

# ObS: se não conseguir fazer o download dos dados usar JNB, converter em csv:
# df.to_csv(r'Path where you want to store the exported CSV file\File Name.csv


## Consultando os Dados no MongoDB

In [80]:
# Desconectar
mystream.disconnect()

In [84]:
# Verificando um documento no collection (qualuqer registro(um twitt))
col.find_one()

## Análise de Dados com Pandas e Scikit-Learn

In [86]:
# criando um dataset com dados retornados do MongoDB
dataset = [{"created_at": item["created_at"], "text": item["text"],} for item in col.find()]

In [ ]:
# Importando o módulo Pandas para trabalhar com datasets em Python
import pandas as pd
pd.__version__

In [ ]:
# Criando um dataframe a partir do dataset 
df = pd.DataFrame(dataset)

In [ ]:
# Imprimindo o dataframe
df

In [ ]:
# Importando o módulo Scikit Learn - contar as vezes que cada palavra aparece no conj de dados  
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
import sklearn
sklearn.__version__

In [ ]:
# Usando o método CountVectorizer para criar uma matriz de documentos
cv = CountVectorizer()
count_matrix = cv.fit_transform(df.text)

In [ ]:
# Contando o número de ocorrências das principais palavras em nosso dataset
word_count = pd.DataFrame(cv.get_feature_names(), columns=["word"])
word_count["count"] = count_matrix.sum(axis=0).tolist()[0]
word_count = word_count.sort_values("count", ascending=False).reset_index(drop=True)
word_count[:50]

OBS: CONEXÃO DO MONGODB COM O COLAB?


Fim 